**Examen Fase II - Curso práctico Bioinformática**

*1. Recuerde cambiar el entorno de ejecución a un entorno de GPU (Runtime menu > Change Runtime Type > choose GPU)*

*2. Lea con calma el enunciado*

*3. Suerte*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install cmake --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: cmake
    Found existing installation: cmake 3.25.2
    Uninstalling cmake-3.25.2:
      Successfully uninstalled cmake-3.25.2


In [3]:
%%bash
wget ftp://ftp.gromacs.org/gromacs/gromacs-2021.tar.gz
tar xfz gromacs-2021.tar.gz
cd /content/gromacs-2021
mkdir /content/gromacs-2021/build
cd /content/gromacs-2021/build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python3: /usr/local/bin/python (found suitable version "3.10.11", minimum required is "3.6") found components: Interpreter 
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Performing Test CFLAGS_WARN_NO_MISSING_FIELD_INITIALIZERS
-- Performing Test CFLAGS_WARN_NO_MISSING_FIELD_INITIALIZERS - Success
-- Performing Test CFLAGS_EXCESS_PREC
-- Performing Test CFLAGS_EXCESS_PREC - Succ

--2023-06-03 17:25:14--  ftp://ftp.gromacs.org/gromacs/gromacs-2021.tar.gz
           => ‘gromacs-2021.tar.gz’
Resolving ftp.gromacs.org (ftp.gromacs.org)... 130.237.11.165, 2001:6b0:1:1191:216:3eff:fec7:6e30
Connecting to ftp.gromacs.org (ftp.gromacs.org)|130.237.11.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /gromacs ... done.
==> SIZE gromacs-2021.tar.gz ... 35061679
==> PASV ... done.    ==> RETR gromacs-2021.tar.gz ... done.
Length: 35061679 (33M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  102K 5m34s
    50K .......... .......... .......... .......... ..........  0%  205K 4m10s
   100K .......... .......... .......... .......... ..........  0%  380K 3m16s
   150K .......... .......... .......... .......... ..........  0%  449K 2m46s
   200K .......... .......... .......... .......... ..........  0%  641K 2m23s
   250K .......... .......... ........

In [ ]:
%%bash
cd /content/gromacs-2021/build
make -j 2
make install
source /usr/local/gromacs/bin/GMXRC

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx

In [ ]:
!export GMXLIB=/usr/local/share/gromacs/top

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx pdb2gmx -f protein.pdb -o protein.gro -p protein.top -i protein.itp -water spc -ignh -ff amber99sb-ildn
gmx editconf -f protein.gro -o protein_newbox.gro -c -d 1.0 -bt cubic
gmx solvate -cp protein_newbox.gro -o protein_solv.gro -p protein.top -cs spc216.gro
gmx grompp -f genion.mdp -c protein_solv.gro -p protein.top -o ions.tpr -maxwarn 2
echo "SOL" > options
echo " " >> options
gmx genion -s ions.tpr -o ions.gro -p protein.top -pname NA -nname CL -neutral < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx grompp -f min.mdp -c ions.gro -p protein.top -o min.tpr
gmx mdrun -v -deffnm min 
gmx grompp -f nve.mdp -c min.gro -p protein.top -o nve.tpr -r min.gro
gmx mdrun -v -deffnm nve -nb gpu -pin on -ntmpi 2 -ntomp 1
gmx grompp -f nvt.mdp -c nve.gro -p protein.top -o nvt.tpr -r nve.gro
gmx mdrun -v -deffnm nvt -nb gpu -pin on -ntmpi 2 -ntomp 1
gmx grompp -f npt.mdp -c nvt.gro -p protein.top -o npt.tpr -r nvt.gro
gmx mdrun -v -deffnm npt -nb gpu -pin on -ntmpi 2 -ntomp 1

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx editconf -f min.gro -o min.pdb -label A
gmx editconf -f nve.gro -o nve.pdb -label A
gmx editconf -f nvt.gro -o nvt.pdb -label A
gmx editconf -f npt.gro -o npt.pdb -label A

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la energia potencial de los archivos de salida "min" 
echo "Potential" > options
echo " " >> options
gmx energy -f min.edr -o min_potential.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Temperature" > options
echo " " >> options
gmx energy -f nve.edr -o temperature_nve.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Temperature" > options
echo " " >> options
gmx energy -f nvt.edr -o temperature_nvt.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Density" > options
echo " " >> options
gmx energy -f npt.edr -o density.xvg -xvg none < options